<a href="https://colab.research.google.com/github/apoorvkh/notebooks/blob/main/Face%20Recognition%20(ArcFace).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
%%bash
git clone https://github.com/peteryuX/arcface-tf2.git arcface
pip install -r arcface/requirements.txt

In [2]:
import requests
import zipfile

def download(file_id, dst_path):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : file_id }, stream = True)

    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            params = { 'id' : file_id, 'confirm' : value }
            response = session.get(URL, params = params, stream = True)
            break

    with open(dst_path, "wb") as f:
        for chunk in response.iter_content(32768):
            if chunk:
                f.write(chunk)

download('1HasWQb86s4xSYy36YbmhRELg9LBmvhvt', 'arc_res50.zip')
with zipfile.ZipFile('arc_res50.zip', 'r') as zip_ref:
    zip_ref.extractall('checkpoints')
!rm arc_res50.zip

# Loading model

In [3]:
from arcface.modules.models import ArcFaceModel
from arcface.modules.utils import set_memory_growth, l2_norm
from skimage.transform import SimilarityTransform
import numpy as np
import tensorflow as tf
import cv2

In [10]:
set_memory_growth()

model = ArcFaceModel(size=112, backbone_type='ResNet50', training=False)
model.load_weights('./checkpoints/arc_res50/e_8_b_40000.ckpt')

CROP_SIZE = (112, 112)

reference_pts = np.float32(np.array([
    [0.341916, 0.4615741],  # left eye
    [0.656533, 0.4598339],  # right eye
    [0.500225, 0.6405054],  # nose
    [0.370975, 0.8246919],  # left mouth
    [0.631516, 0.8232509]   # right mouth
]) * CROP_SIZE)

def warp_to_face(img, face_pts):
    tform = SimilarityTransform()
    tform.estimate(face_pts, reference_pts)
    T = tform.params[:2]
    return cv2.warpAffine(img, T, CROP_SIZE)

# img : H x W x 3  matrix(BGR, np.uint8)
# face_pts : List of 5 x 2 matrices (np.float32)
def compute_embeddings(img, face_pts):
    faces = [warp_to_face(img, pts) for pts in face_pts]
    faces = np.float32(faces) / 255.
    emb = l2_norm(model(faces))
    return emb

# Inference

In [11]:
img = cv2.imread('arcface/data/BruceLee.jpg')
face_pts = [reference_pts, reference_pts] # placeholder

embedding = compute_embeddings(img, face_pts)

print(embedding.shape)

(2, 512)
